In [ ]:
!pip install torch llama-index transformers

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
import openai
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import os
openai.api_key = os.environ['OPENAI_API_KEY']

In [ ]:
coqa_dataset = load_dataset("stanfordnlp/coqa")

In [ ]:
from llama_index.core import StorageContext, Document, Settings, KnowledgeGraphIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.llms.openai import OpenAI

In [ ]:
documents = [Document(text=doc) for doc in coqa_dataset["train"].to_pandas()['story']][:2000]

In [ ]:
llm = OpenAI(temperature=0, model="gpt-3.5-turbo-instruct")
Settings.llm = llm
Settings.chunk_size = 512

In [ ]:
graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)

In [ ]:
from transformers import pipeline

triplet_extractor = pipeline(
    "text2text-generation",
    model="Babelscape/rebel-large",
    tokenizer="Babelscape/rebel-large",
    # comment this line to run on CPU
    device="cuda:0",
)


def extract_triplets(input_text):
    text = triplet_extractor.tokenizer.batch_decode(
        [
            triplet_extractor(
                input_text, return_tensors=True, return_text=False
            )[0]["generated_token_ids"]
        ]
    )[0]

    triplets = []
    relation, subject, relation, object_ = "", "", "", ""
    text = text.strip()
    current = "x"
    for token in (
        text.replace("<s>", "")
        .replace("<pad>", "")
        .replace("</s>", "")
        .split()
    ):
        if token == "<triplet>":
            current = "t"
            if relation != "":
                triplets.append(
                    (subject.strip(), relation.strip(), object_.strip())
                )
                relation = ""
            subject = ""
        elif token == "<subj>":
            current = "s"
            if relation != "":
                triplets.append(
                    (subject.strip(), relation.strip(), object_.strip())
                )
            object_ = ""
        elif token == "<obj>":
            current = "o"
            relation = ""
        else:
            if current == "t":
                subject += " " + token
            elif current == "s":
                object_ += " " + token
            elif current == "o":
                relation += " " + token

    if subject != "" and relation != "" and object_ != "":
        triplets.append((subject.strip(), relation.strip(), object_.strip()))

    return triplets

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/344 [00:00<?, ?B/s]

In [ ]:
'''index = KnowledgeGraphIndex.from_documents(
    documents[:100],
    max_triplets_per_chunk=5,
    kg_triplet_extract_fn=extract_triplets,
    storage_context=storage_context,
    include_embeddings=True,
    show_progress=True,
)'''

index = KnowledgeGraphIndex.from_documents(
    documents[:100],
    max_triplets_per_chunk=5,
    storage_context=storage_context,
    show_progress=True,
    include_embeddings=True,
)

Parsing nodes:   0%|          | 0/100 [00:00<?, ?it/s]

Processing nodes:   0%|          | 0/102 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

...

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
query_engine = query_engine = index.as_query_engine(
            include_text=False,
            response_mode="tree_summarize",
            retriever_mode="embedding",
            similarity_top_k=3,
            verbose=True,
            graph_store_query_depth=3,
        )

In [ ]:
questions = [row[0] for row in coqa_dataset["train"].to_pandas()['questions'][:2000]]
answers = [row['input_text'][0] for row in coqa_dataset["train"].to_pandas()['answers'][:2000]]

In [ ]:
response = query_engine.query(questions[0])

Extracted keywords: ['opened', 'formally', 'Vat']
KG context:
The following are knowledge sequence in max depth 3 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Vatican library', 'Was established in', '1475')
('Vatican library', 'Is open to', 'Anyone')
('The legion of honour', 'Established in', '1802')


In [ ]:
triplets = response.source_nodes[0].metadata["kg_rel_texts"]

In [ ]:
from collections import Counter

total_nodes = set()
for t in triplets:
  triplet_list = t[1:-1].split("'")
  total_nodes.add(triplet_list[1])
  total_nodes.add(triplet_list[-2])

print(total_nodes)

{'1802', 'The legion of honour', 'Anyone', '1475', 'Vatican library'}


In [ ]:
rel_map = index.graph_store.get_rel_map(list(total_nodes), 3)

In [ ]:
full_triplet_list = []
for node, triplet_list in rel_map.items():
  full_triplet_list.extend(triplet_list)

print(full_triplet_list)

[['The legion of honour', 'Full name', 'National order of the legion of honour'], ['The legion of honour', 'Established in', '1802'], ['The legion of honour', 'Motto', 'Honour and fatherland'], ['The legion of honour', 'Seat', "Next to the musée d'orsay"], ['The legion of honour', 'Divided into', 'Five degrees'], ['Vatican library', 'Is located in', 'Vatican city'], ['Vatican library', 'Was established in', '1475'], ['Vatican library', 'Contains', 'Significant collections'], ['Vatican library', 'Is open to', 'Anyone']]


In [ ]:
triplet_embeddings = []
for triplet in full_triplet_list:
  triplet_embeddings.append(index.index_struct.embedding_dict[str(tuple(triplet))])

In [ ]:
!pip install chromadb

In [ ]:
!pip install llama-index-vector-stores-chroma
!pip install llama-index-embeddings-huggingface

In [ ]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core.extractors import BaseExtractor
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import set_global_service_context, PromptHelper, ServiceContext, VectorStoreIndex
from typing import List, Dict

In [ ]:
# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./chroma_db")

# create collection
chroma_collection = db.get_or_create_collection("quickstart")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
doc_index = VectorStoreIndex.from_documents(documents, show_progress=True, storage_context=storage_context)

Parsing nodes:   0%|          | 0/2000 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/12 [00:00<?, ?it/s]

In [ ]:
from llama_index.core.schema import QueryBundle, NodeWithScore, TextNode

doc_retriever = doc_index.as_retriever(
    similarity_top_k=5,
)

all_nodes = set()
for emb in triplet_embeddings:
  node_list = doc_retriever.retrieve(QueryBundle(query_str="", embedding=emb))
  all_nodes.update({node.text for node in node_list})

node_set = [NodeWithScore(node=TextNode(text=chunk)) for chunk in all_nodes]

In [ ]:
!pip install llama-index-postprocessor-flag-embedding-reranker

In [ ]:
from llama_index.postprocessor.flag_embedding_reranker import FlagEmbeddingReranker

reranker = FlagEmbeddingReranker(
    top_n=5,
    model="BAAI/bge-reranker-large",
    use_fp16=False
)

In [ ]:
query_bundle = QueryBundle(query_str=questions[0])
ranked_nodes = reranker._postprocess_nodes(node_set, query_bundle)

In [ ]:
ranked_nodes = list(filter(lambda node: (node.score > 0), ranked_nodes))

In [ ]:
ranked_nodes

[NodeWithScore(node=TextNode(id_='6d5eb03b-a24d-4320-8b3b-c37e5f79a82b', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='The Vatican Apostolic Library (), more commonly called the Vatican Library or simply the Vat, is the library of the Holy See, located in Vatican City. Formally established in 1475, although it is much older, it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts. It has 75,000 codices from throughout history, as well as 1.1 million printed books, which include some 8,500 incunabula. \n\nThe Vatican Library is a research library for history, law, philosophy, science and theology. The Vatican Library is open to anyone who can document their qualifications and research needs. Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail. \n\nIn March 2014, the Vatican Library began